In [1]:
from google.colab import drive
drive.mount('/content/drive')

shared_folder_path = '/content/drive/MyDrive/ds_project_dataset'

!cp -r "$shared_folder_path" .

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from multiprocessing import Pool
from functools import partial

In [3]:
img_size = 224

In [4]:
def load_image(image_path, resize_shape=None):
    img = cv2.imread(image_path)
    if resize_shape is not None:
        img = cv2.resize(img, resize_shape)
    return img

def load_images_parallel(image_paths, n_jobs=4, resize_shape=None):
    with Pool(processes=n_jobs) as pool:
        func = partial(load_image, resize_shape=resize_shape)
        images = pool.map(func, image_paths)
    return np.array(images)

In [5]:
df = pd.read_csv('ds_project_dataset/articles.csv')

In [6]:
selected_categories = ['Garment Upper body', 'Underwear', 'Socks & Tights', 'Garment Lower body', 'Shoes', 'Garment Full body', 'Bags']
df = df[df['product_group_name'].isin(selected_categories)]

In [7]:
selected_categories = ['undefined', 'Unknown']
df = df[~df['perceived_colour_master_name'].isin(selected_categories)]

In [8]:
df['perceived_colour_master_name'] = df['perceived_colour_master_name'].replace(['Khaki green', 'Yellowish Green', 'Bluish Green', 'Lilac Purple'], ['Green', 'Green', 'Green', 'Purple'])

In [9]:
selected_categories = ['Other pattern', 'Unknown', 'Other structure']
df = df[~df['graphical_appearance_name'].isin(selected_categories)]

In [10]:
selected_categories = ['Ladieswear', 'Menswear', 'Divided']
df = df[df['index_group_name'].isin(selected_categories)]

In [11]:
df['subfolder'] = '0' + df['article_id'].astype(str).str[:2]
df['image_path'] = 'ds_project_dataset/images/' + df['subfolder'].str.strip() + '/0' + df['article_id'].astype(str) + '.jpg'

<ipython-input-11-1d0273c0ab80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subfolder'] = '0' + df['article_id'].astype(str).str[:2]
<ipython-input-11-1d0273c0ab80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_path'] = 'ds_project_dataset/images/' + df['subfolder'].str.strip() + '/0' + df['article_id'].astype(str) + '.jpg'


In [12]:
import os

valid_images = []
for index, row in tqdm(df[:11000].iterrows(), total=len(df[:11000])):
    if os.path.isfile(row['image_path']):
        valid_images.append(row)

df_valid_images = pd.DataFrame(valid_images)

100%|██████████| 11000/11000 [02:39<00:00, 69.01it/s] 


In [13]:
df_valid_images.shape

(10897, 27)

In [14]:
images = load_images_parallel(df_valid_images['image_path'].values, n_jobs=4, resize_shape=(img_size, img_size))

In [15]:
import tensorflow as tf

In [21]:
resnet_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', pooling='avg')

# Define the image preprocessing pipeline
preprocess_input = tf.keras.applications.resnet50.preprocess_input

embeddings = []
for image in tqdm(images):
    # Add an extra batch dimension to the image
    image = np.expand_dims(image, axis=0)

    # Pass the image through the ResNet50 model
    features = resnet_model.predict(image)

    # Extract the embeddings from the output of the model
    embeddings.append(features.squeeze())

  0%|          | 0/10897 [00:00<?, ?it/s]

1/1 [==============================] - 9s 9s/step


  0%|          | 1/10897 [00:09<28:24:59,  9.39s/it]

1/1 [==============================] - 0s 26ms/step


  0%|          | 3/10897 [00:09<7:30:56,  2.48s/it] 

...

100%|█████████▉| 10895/10897 [19:31<00:00,  9.77it/s]

1/1 [==============================] - 0s 44ms/step


100%|█████████▉| 10896/10897 [19:31<00:00,  9.19it/s]

1/1 [==============================] - 0s 54ms/step


100%|██████████| 10897/10897 [19:31<00:00,  9.30it/s]


In [41]:
new_df = pd.DataFrame(df_valid_images['article_id'])
new_df = new_df.join(df_valid_images['perceived_colour_master_name'])
new_df = new_df.join(df_valid_images['product_group_name'])
new_df = new_df.join(df_valid_images['graphical_appearance_name'])
new_df = new_df.join(df_valid_images['index_group_name'])

In [42]:
new_df['embedding'] = embeddings

In [44]:
new_df.to_parquet('embeddings_df1.parquet')